# Albumentationを用いたData Augmentation  

Dataset：  
https://www.kaggle.com/iafoss/256x256-images  
iafoss氏の256クロップ済みデータを用いる。  

実際にPytorchで学習を回すことを意識し、毎回Datasetクラスに載せて読み込む。(但し、いらない部分はひたすら削ってある)  
なお、今回は可視化が目的のため、Normalizeを行わない。  

Albumentations Documentation :  
https://albumentations.ai/docs/  
分からない関数や引数があった場合は、このサイトの検索窓にそのまま入力すれば大体解決します。

In [ ]:
import os
import pandas as pd
import numpy as np
import random
import cv2

import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

from pylab import rcParams
rcParams['figure.figsize'] = 20,10

import torch
from torch.utils.data import Dataset,DataLoader
import albumentations as A

In [ ]:
#iafoss dataset
TRAIN = '../input/hubmap-256x256/train/'
MASKS = '../input/hubmap-256x256/masks/'
LABELS = '../input/hubmap-kidney-segmentation/train.csv'

# Dataset  

In [ ]:
class HuBMAPDataset(Dataset):
    def __init__(self,tfms=None):
        ids = pd.read_csv(LABELS).id.values
        self.fnames = [fname for fname in os.listdir(TRAIN) if fname.split('_')[0] in ids]
        self.tfms = tfms
        
    def __len__(self):
        return len(self.fnames)
    
    def __getitem__(self, idx):
        fname = self.fnames[idx]
        img = cv2.cvtColor(cv2.imread(os.path.join(TRAIN,fname)), cv2.COLOR_BGR2RGB)
        mask = cv2.imread(os.path.join(MASKS,fname),cv2.IMREAD_GRAYSCALE)
        if self.tfms is not None:
            augmented = self.tfms(image=img,mask=mask)
            img,mask = augmented['image'],augmented['mask']
        return img,mask

In [ ]:
#画像表示のための関数
good_img_index = [1,5,7,8]
def show_images(tfms):
    dataset_show = HuBMAPDataset(tfms)
    for i in range(2):
        f, axarr = plt.subplots(1,4)
        for p in range(2):
            idx = i*2+p
            img, mask = dataset_show[good_img_index[idx]]
            img = img.astype(np.uint8)
            axarr[p*2].imshow(img)
            axarr[p*2+1].imshow(mask)

In [ ]:
#transformなし(元画像)
show_images(tfms=None)

# Resize  
基本的にはAugmentationの最後に行う。

In [ ]:
tfms = A.Compose([
    A.Resize(512, 512),
])

show_images(tfms=tfms)

# Flip  

In [ ]:
#水平方向
tfms = A.Compose([
    A.HorizontalFlip(p=1.), #確か初期値は0.5
])

show_images(tfms=tfms)

In [ ]:
#垂直方向
tfms = A.Compose([
    A.VerticalFlip(p=1.),
])

show_images(tfms=tfms)

# Rotate

In [ ]:
tfms = A.Compose([
    A.RandomRotate90(p=1.), #0,90,180,270°方向にランダムにRotate
])

show_images(tfms=tfms)

# Transpose  
転置。  
対角成分を基準にして鏡写しに入れ替わる。  

In [ ]:
tfms = A.Compose([
    A.Transpose(p=1.),
])

show_images(tfms=tfms)

# アフィン変換(Affine Transform)  
Albumentation内では、"ShiftScaleRotate"。  
画像の拡大縮小、回転、平行移動などをまとめて行う。  

参考：  
https://imagingsolution.blog.fc2.com/blog-entry-284.html  

In [ ]:
tfms = A.Compose([
    A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=15, p=0.9, 
                         border_mode=cv2.BORDER_REFLECT),
])

show_images(tfms=tfms)

In [ ]:
#ボーダー(外挿)なし
tfms = A.Compose([
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, border_mode=0, p=0.85),
])

show_images(tfms=tfms)

# Brightness  

In [ ]:
tfms = A.Compose([
    A.RandomBrightness(limit=0.2, p=0.75),
])

show_images(tfms=tfms)

# Contrast  

In [ ]:
tfms = A.Compose([
    A.RandomContrast(limit=0.2, p=0.75),
])

show_images(tfms=tfms)

# Blur(ぼかし)系

In [ ]:
tfms = A.Compose([
    A.MotionBlur(blur_limit=5),
])

show_images(tfms=tfms)

In [ ]:
tfms = A.Compose([
    A.MedianBlur(blur_limit=5),
])

show_images(tfms=tfms)

In [ ]:
tfms = A.Compose([
    A.GaussianBlur(blur_limit=5),
])

show_images(tfms=tfms)

# Noise系  

In [ ]:
tfms = A.Compose([
    A.GaussNoise(var_limit=(5.0, 30.0),p=1.),
])

show_images(tfms=tfms)

In [ ]:
tfms = A.Compose([
    A.ISONoise(p=1.),
])

show_images(tfms=tfms)

# 歪み系(Distortion)

In [ ]:
tfms = A.Compose([
    A.OpticalDistortion(distort_limit=1.0),
])

show_images(tfms=tfms)

In [ ]:
tfms = A.Compose([
    A.GridDistortion(num_steps=5, distort_limit=1.),
])

show_images(tfms=tfms)

In [ ]:
tfms = A.Compose([
    A.ElasticTransform(alpha=3),
])

show_images(tfms=tfms)

# ヒストグラム平坦化(Histogram Flattening)  

http://labs.eecs.tottori-u.ac.jp/sd/Member/oyamada/OpenCV/html/py_tutorials/py_imgproc/py_histograms/py_histogram_equalization/py_histogram_equalization.html

In [ ]:
tfms = A.Compose([
    A.CLAHE(clip_limit=4.0, p=0.7),
])

show_images(tfms=tfms)


# Cutout  
Albumentationsには2種類のカットアウトが実装されており、現在は"Cutout"関数ではなく、"CoarseDropout"関数を使用することが推奨されている。    
前者ではマスク画像へのCutoutの適用ができないことに注意。  
Albumentations implements two types of Cutouts, and it is now recommended to use the "CoarseDropout" function instead of the "Cutout" function.      
Note that the former does not allow you to apply Cutout to a masked image.  

In [ ]:
#従来のCutout,現在のバージョンではこの関数を使うことは推奨されていない(DeprecationWarning)
#セグメント画像にCutoutは適用できない。(今回のコンペではこっちは使わない方が無難である)
"""
Conventional Cutout function, in the current version it is not recommended to use this function (DeprecationWarning)
Cutout cannot be applied to mask images.
"""
tfms = A.Compose([
    A.Cutout(max_h_size=50, max_w_size=50, num_holes=2, p=1.),
])

show_images(tfms=tfms)

In [ ]:
#そこで、現在推奨されているCutoutは、"CoarseDropout"関数で利用できる。
#引数mask_fill_value=0とすれば、マスク画像にもカットアウトを適用できる。
"""
Cutout, which is currently recommended, is available in the "CoarseDropout" function.
If you set the argument mask_fill_value=0, you can apply the cutout to the masked image as well.
"""
tfms = A.Compose([
    A.CoarseDropout(max_holes=2,min_holes=1,max_height=50,max_width=50,min_height=20,min_width=20,mask_fill_value=0, p=1.),
])

show_images(tfms=tfms)

# HSV色空間

https://ja.wikipedia.org/wiki/HSV%E8%89%B2%E7%A9%BA%E9%96%93

In [ ]:
tfms = A.Compose([
    A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=10, p=1.),
])

show_images(tfms=tfms)

# どれか一つを適用する "OneOf"  
似たような処理でまとめることが多い  

In [ ]:
tfms = A.Compose([
    A.OneOf([
        A.MotionBlur(blur_limit=5),
        A.MedianBlur(blur_limit=5),
        A.GaussianBlur(blur_limit=5),
        A.GaussNoise(var_limit=(5.0, 30.0)),
    ], p=0.7),
])

show_images(tfms=tfms)


# すごいやつ  
Melanomaコンペ1st placeのData Augmentation.(今回のコンペに向いているというわけでは無いので注意)  
処理にもすごい時間がかかるのでKaggle Notebookでは非現実的…(GPU上でAugmentationできるdaliやkorniaならあるいは…)  
https://www.kaggle.com/haqishen/1st-place-soluiton-code-small-ver  

In [ ]:
tfms = A.Compose([
    A.Transpose(p=0.5),
    A.VerticalFlip(p=0.5),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightness(limit=0.2, p=0.75),
    A.RandomContrast(limit=0.2, p=0.75),
    A.OneOf([
        A.MotionBlur(blur_limit=5),
        A.MedianBlur(blur_limit=5),
        A.GaussianBlur(blur_limit=5),
        A.GaussNoise(var_limit=(5.0, 30.0)),
    ], p=0.7),

    A.OneOf([
        A.OpticalDistortion(distort_limit=1.0),
        A.GridDistortion(num_steps=5, distort_limit=1.),
        A.ElasticTransform(alpha=3),
    ], p=0.7),

    A.CLAHE(clip_limit=4.0, p=0.7),
    A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=10, p=0.5),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, border_mode=0, p=0.85),
    A.Resize(256, 256),
    A.Cutout(max_h_size=int(256 * 0.375), max_w_size=int(256 * 0.375), num_holes=1, p=0.7),    
    #A.Normalize()
])

show_images(tfms=tfms)